# Post-process: Smoothing and Reclassify Classes

## Background
In real case studies machine learning may not lead to desired classification maps due to factors including limited training data and limited performance of the model for prediction. Therefore, post-processing is widely applied to the predicted classification results, based on assumptions or existing knowlegde of the ground truth. Commonly applied post-processing includes manual editing, filtering, reclassification and class merging, etc.   

## Description
In this notebook we will apply post-processing to the land cover maps we produced through the previous notebook. We will use external layers that contain reliable information on centain classes and/or have higher-spatial resolution to reclassify classes that may be misclassified by the random forest classifier. These external layers have been prepared and uploaded into 'Data/' folder. We'll also conduct a median filtering to reduce the 'salt and pepper' effect resulted from pixel-based classification. This notebook will demonstrate how to do implement these post-processings and visualise the comparison before and after the post-processing. The steps are as follows:
1. Load the external layers and land cover maps produced at the testing locations
2. Majority filtering of the maps to reduce salt-and-pepper effects
3. Apply customised reclassification rules using the external layers
4. Save the results to disk as COGs


To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load Packages

In [ ]:
%matplotlib inline
import datacube
import warnings
import numpy as np
import geopandas as gpd
import xarray as xr
import rioxarray
from rasterio.enums import Resampling
from datacube.utils.cog import write_cog
from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.plotting import rgb
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.coastal import get_coastlines
from skimage.morphology import binary_dilation,disk
from skimage.filters.rank import modal
from odc.algo import xr_reproject
import matplotlib.pyplot as plt
import subprocess
from matplotlib.colors import ListedColormap,BoundaryNorm
from matplotlib.patches import Patch

## Analysis parameters
* `prediction_maps_path`: A list of file paths and names of the classification maps produced in the previous notebook.
* `rgb_images_path`: A list of file paths and names of the true colour images at the prediction locations exported in the previous notebook.
* `dict_map`: A dictionary map of class names corresponding to pixel values.
* `output_crs`: Coordinate reference system for output raster files.

In [ ]:
prediction_maps_path=['Results/Mozambique_land_cover_prediction_2021_location_'+str(i)+'.tif' for i in range(3)] # list of prediction map files
rgb_images_path=['Results/Mozambique_satellite_image_2021_location_'+str(i)+'.tif' for i in range(3)] # list of rgb images at the prediction locations
dict_map={'Tree crops':11,'Field crops':12,'Forest plantations':21,'Grassland':31,
                 'Aquatic or regularly flooded herbaceous vegetation':41,'Water body':44,
                 'Settlements':51,'Bare soils':61,'Mangrove':70,'Mecrusse':71,
                'Broadleaved (Semi-) evergreen forest':72,'Broadleaved (Semi-) deciduous forest':74,'Mopane':75} # a dictionary of pixel value for each class
output_crs='epsg:32736' # WGS84/UTM Zone 36S

## External Layers
The country boundary shapefile and a few external layers were sourced and prepared in the 'Data/' folder, which are helpful to provide information on specific classes, e.g. Settlementss and Water Body. The data includes:
* `country_boundary_shp`: Country boundary shapefile.
* `hand_raster`: Hydrologically adjusted elevations, i.e. Height Above the Nearest Drainage (hand) derived from the [MERIT Hydro dataset](https://developers.google.com/earth-engine/datasets/catalog/MERIT_Hydro_v1_0_1#description).
* `river_network_shp`: OSM river network shapefile. The OSM layers were sourced from the [Humanitarian OpenStreetMap Team (HOT)](https://data.humdata.org/organization/hot) website.
* `road_network_shp`: OSM road network shapefile.
* `google_building_raster`: A rasterised layer of [Google Open Building polygons](https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_Research_open-buildings_v2_polygons), which consist of outlines of buildings derived from high-resolution 50 cm satellite imagery. As there are many polygons in the original vector layer, we rasterised the layer to 10 m resolution to reduce disk storage and memory required for processing.
* `wsf2019_raster`: 2019 [World Settlement Footprint (WSF) layer](https://gee-community-catalog.org/projects/wsf/), a 10m resolution binary mask outlining the extent of human settlements globally derived by means of 2019 multitemporal Sentinel-1 and Sentinel-2 imagery.

> Note: In this notebook we have made the data prepared for you to run through the demonstration. If you would like to apply it to your own project, you may need to spend some time sourcing the datasets and do some pre-processing if needed, e.g. clipping to your study area, filtering, rasterisation or vectorisation. Alternatively you can revise this notebook depending on your data format.

In [ ]:
country_boundary_shp='Data/Mozambique_boundary.shp' # country boundary shapefile
river_network_shp='Data/hotosm_moz_waterways_lines_filtered.shp' # OSM river network data
road_network_shp='Data/hotosm_moz_roads_lines_filtered.shp' # OSM road network data
google_building_raster='Data/GoogleBuildingLayer_Mozambique_rasterised.tif' # rasterised google bulding mask layer
hand_raster='Data/hand_Mozambique_UInt16.tif' # Hydrologically adjusted elevations, i.e. height above the nearest drainage (hand)
wsf2019_raster='Data/WSF2019_v1_Mozambique_clipped.tif' # 2019 World Settlement Footprint layer

As the raster layers cover the entire country, they can be too large to be loaded to your Sandbox memory (especially if you are using the default instance) and used for analysis. Therefore, in this notebook we first clipped the layers to the extents of the test prediction locations using GDAL commands. We first create the shapefiles of the extents of the predicted maps using [`gdaltindex`](https://gdal.org/programs/gdaltindex.html), then clip the raster layers using [`gdalwarp`](https://gdal.org/programs/gdalwarp.html).

In [ ]:
google_building_tiles=[google_building_raster[:-4]+'_location_'+str(i)+'.tif' for i in range(len(prediction_maps_path))] # clipped google bulding mask layer at test locations
hand_raster_tiles=[hand_raster[:-4]+'_location_'+str(i)+'.tif' for i in range(len(prediction_maps_path))] # clipped hand layer at test locations
wsf2019_raster_tiles=[wsf2019_raster[:-4]+'_location_'+str(i)+'.tif' for i in range(len(prediction_maps_path))] # clipped WSF 2019 layer at test locations
for i in range(len(prediction_maps_path)):
    tile_shp='Results/Mozambique_tile'+'_location_'+str(i)+'.shp' # output region extents
    subprocess.run(['gdaltindex',tile_shp,prediction_maps_path[0]])
    subprocess.run(['gdalwarp','-cutline',tile_shp,'-crop_to_cutline', google_building_raster,google_building_tiles[i]])
    subprocess.run(['gdalwarp','-cutline',tile_shp,'-crop_to_cutline', hand_raster,hand_raster_tiles[i]])
    subprocess.run(['gdalwarp','-cutline',tile_shp,'-crop_to_cutline', wsf2019_raster,wsf2019_raster_tiles[i]])

## Load layers
First let's load the land cover maps and true colour images generated from the previous notebook:

In [ ]:
# import land cover map of 2021 and reproject
prediction_maps=[]
rgb_images=[]
for i in range(len(prediction_maps_path)):
    lc_map=rioxarray.open_rasterio(prediction_maps_path[i]).astype(np.uint8).squeeze()
    prediction_maps.append(lc_map)
    
    rgb_image=rioxarray.open_rasterio(rgb_images_path[i])
    rgb_images.append(rgb_image)

We then load other layers. The OSM road network layer contains multi-lines with various surface attributes. We'll select some major road types and buffer them by 10 metres:

In [ ]:
# import OSM road network data and reproject
road_network=gpd.read_file(road_network_shp).to_crs(output_crs) 
road_network=road_network.loc[road_network['surface'].isin(['asphalt', 'paved', 'compacted', 'cobblestone', 
                                                             'concrete', 'metal', 'paving_stones', 
                                                             'paving_stones:30'])] # select road network by attributes
road_network.geometry=road_network.geometry.buffer(10) # buffer the road network by 10m

Similarly we load and select major waterways from the OSM river network layer:

In [ ]:
river_network=gpd.read_file(river_network_shp).to_crs(output_crs) # import OSM river network data and reproject
river_network=river_network.loc[river_network['waterway'].isin(['canal','river'])] # select river network by attribute

## Morphological filtering
Now we start the post-processing by applying a majority filtering, a commonly applied step to reduce salt-and-pepper noise typical in pixel-based classification. To demonstrate each post-processing step we will process the first prediction map, then put the steps together in an iterative loop to process all prediction. The majority filtering is applied within each local window with a given footprint. Here we use a disk with radius of two pixels. It is advised that you adjust the footprint depending on your prediction results and desired effects.

In [ ]:
i=0
lc_map=prediction_maps[i]
# convert to numpy array
np_lc_map=lc_map.squeeze().to_numpy()
# mode filtering for a smoother classification map
np_lc_map_postproc=modal(np_lc_map,footprint=disk(2),mask=np_lc_map!=0)

We can plot and compare the maps before and after filtering:

In [ ]:
# reconstruct dataArray
lc_map_postproc=xr.DataArray(data=np_lc_map_postproc,dims=['y','x'],
                             coords={'y':lc_map.y.to_numpy(), 'x':lc_map.x.to_numpy()})
# display colour for each class value
colours = {11:'gold', 12:'yellow', 21:'darkred',31:'bisque',41:'lightgreen',44:'blue',51:'black',61:'gray',
           70:'red',71:'steelblue',72:'blueviolet',74:'green',75:'chocolate'}

fig, axes = plt.subplots(1, 2, figsize=(24, 8))

# Plot classified image before filtering
prediction_values=np.unique(lc_map)
cmap=ListedColormap([colours[k] for k in prediction_values])
norm = BoundaryNorm(list(prediction_values)+[np.max(prediction_values)+1], cmap.N)
lc_map.plot.imshow(ax=axes[0], 
                   cmap=cmap,
                   norm=norm,
                   add_labels=True, 
                   add_colorbar=False,
                   interpolation='none')

# Plot classified image after filtering
lc_map_postproc.plot.imshow(ax=axes[1], 
                   cmap=cmap,
                   norm=norm,
                   add_labels=True, 
                   add_colorbar=False,
                   interpolation='none')
# Remove axis on right plot
axes[1].get_yaxis().set_visible(False)
# add colour legend
patches_list=[Patch(facecolor=colour) for colour in colours.values()]
axes[1].legend(patches_list, list(dict_map.keys()),loc='upper center', ncol =3, bbox_to_anchor=(0.5, -0.1))
# Add plot titles
axes[0].set_title('Classified Image Before Majority Filtering')
axes[1].set_title('Classified Image After Majority Filtering')

# make a copy of intermediate result
lc_map_filtered=lc_map_postproc.copy()

## Apply rules using external layers
Before applying reclassification using other layers, one thing to note for raster-based calculation is to make sure all rasters are in the same spatial reference and align with each other. Here we extract the `geobox` of the land cover map, which defines the location and resolution of the grid data including spatial reference. We will use it to reproject other layers to be aligned with the land cover map.

In [ ]:
ds_geobox=lc_map.geobox

### Recalssify Water body

Now let's reclassify some classes using the external layers. First, we reclassify all pixels classified as Water body and occuring at bottom of watersheds, i.e. height above nearest drainage below 45 m, or falling within OSM river networks as Water body class:

In [ ]:
# load and reproject hand layer
hand=xr.open_dataset(hand_raster_tiles[i],engine="rasterio").squeeze()
hand=xr_reproject(hand, ds_geobox, resampling="average")
# convert to numpy array
np_hand=hand.to_array().squeeze().to_numpy()
# rasterise river network layer
river_network_mask=xr_rasterize(gdf=river_network,
                                  da=lc_map.squeeze(),
                                  transform=ds_geobox.transform,
                                  crs=output_crs)
# convert to numpy array
np_river_network_mask=river_network_mask.to_numpy()
# apply the rule
np_lc_map_postproc[((np_lc_map==dict_map['Water body'])&(np_hand<=45))
                   |(np_river_network_mask==1)]=dict_map['Water body']

We observed that some coastal water was not misclassified, possibly due to insufficient training samples for this type of water. To enhance water body and distinguish it from other classes, we calculate the Modified Normalised Difference Water Index (MNDWI), which is calculated using green and short-wave infrared bands. More information on MNDWI can be found in [this paper](https://www.tandfonline.com/doi/abs/10.1080/01431160600589179). To calculate MNDWI We extract the bounding box of the land cover map to query annual geomedian of Sentinel-2 image, then calcualte MNDWI:

In [ ]:
bbox=ds_geobox.extent.boundingbox
dc = datacube.Datacube(app='s2_geomedian')
query_geomedian= {
    'time': ('2021'),
    'x': (bbox[0],bbox[2]),
    'y': (bbox[1],bbox[3]),
    'resolution':(-10, 10),
    'crs':output_crs,
    'output_crs': output_crs,
    'measurements':['green','swir_1']
}
ds_geomedian = dc.load(product="gm_s2_annual", **query_geomedian)
ds_MNDWI = calculate_indices(ds=ds_geomedian, index='MNDWI', satellite_mission='s2',drop=True).squeeze()

MNDWI has higher values at water body than the other classes, so here we use a threshold of 0 to derive a water body mask and apply it. You can also fine-tune this threshold depending on your area and observation:

In [ ]:
# reassign water using NDWI calculated from annual S2 geomedian
np_MNDWI=ds_MNDWI['MNDWI'].to_numpy()
np_lc_map_postproc[np_MNDWI>=0]=dict_map['Water body']

### Reclassify Settlements
We then assign pixels overlapping google building polygons or WSF 2019 mask as Settlements:

In [ ]:
# load and reproject google buildings raster
google_buildings=xr.open_dataset(google_building_tiles[i],engine="rasterio").squeeze()
google_buildings_mask=xr_reproject(google_buildings, ds_geobox, resampling="average")
# convert to numpy array
np_google_buildings_mask=google_buildings_mask.to_array().squeeze().to_numpy()

# load and reproject WSF 2019 layer
wsf2019=xr.open_dataset(wsf2019_raster_tiles[i],engine="rasterio").squeeze()
wsf2019=xr_reproject(wsf2019, ds_geobox, resampling="nearest")
# convert to numpy array
np_wsf2019=wsf2019.to_array().squeeze().to_numpy()
# apply rule
np_lc_map_postproc[(np_google_buildings_mask==1)|(np_wsf2019==255)]=dict_map['Settlements']

In addition, we reclassify pixels overlapping OSM road network as Settlements class:

In [ ]:
# rasterise road network layer
road_network_mask=xr_rasterize(gdf=road_network,
                              da=lc_map.squeeze(),
                              transform=ds_geobox.transform,
                              crs=output_crs)
# convert to numpy
np_road_network_mask=road_network_mask.to_numpy()
# apply the rule
np_lc_map_postproc[np_road_network_mask==1]=dict_map['Settlements']

### Reclassify regularly flooded vegetation
We assume that regularly flooded vegetation too close (e.g. within 50m) to Settlements are likely misclassified, so we reclassify them as Field crops instead:

In [ ]:
# buffer Settlements areas
urban_buffered=binary_dilation(np_lc_map==dict_map['Settlements'],footprint=disk(5))
# apply rule
np_lc_map_postproc[(urban_buffered==1)&(np_lc_map==dict_map['Aquatic or regularly flooded herbaceous vegetation'])]=dict_map['Field crops']

### Reclassify Mangrove
We also observed that some inland forests are misclassified as Mangrove, which is unlikely as Mangrove grows in coastal areas. Therefore, we reclassify Mangrove pixels further than a pre-defined distance (e.g. 50 km) of the coastline as Forest Plantation. To do that we use the function `get_coastlines` to query the 2021 annual coastline product available in DE Africa. More information about the coastline product can be found in the [notebook](https://docs.digitalearthafrica.org/en/latest/sandbox/notebooks/Datasets/Coastlines.html?highlight=get_coastlines#id1).

In [ ]:
# import mozambique boundary and get bounding box
country_boundary=gpd.read_file(country_boundary_shp).to_crs(output_crs)
# load coastline layer and buffer
shorelines_gdf = get_coastlines(country_boundary.bounds.iloc[0],crs=output_crs,layer='shorelines').to_crs(output_crs)
# select only 2021
shorelines_gdf_2021=shorelines_gdf[shorelines_gdf['year']=='2021'] 
# buffer the road network by 50km
shorelines_gdf_2021.geometry=shorelines_gdf_2021.geometry.buffer(50000) 
# rasterise layer
shorelines_2021_mask=xr_rasterize(gdf=shorelines_gdf_2021,da=lc_map.squeeze(),
                                  transform=ds_geobox.transform,crs=output_crs) 

# load and clip shoreline mask layers
shorelines_2021_mask_tile=xr_reproject(shorelines_2021_mask, ds_geobox, resampling="nearest") 
# convert to numpy
np_shorelines_2021_mask=shorelines_2021_mask_tile.squeeze().to_numpy()
# apply rule
np_lc_map_postproc[(np_shorelines_2021_mask==0)&(np_lc_map==dict_map['Mangrove'])]=dict_map['Forest plantations']

### Plot the results
We can plot the maps to see a comparison before and after applying the above rules using the external layers:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(24, 8))

# Plot classified image before applying rules
prediction_values=np.unique(lc_map_filtered)
cmap=ListedColormap([colours[k] for k in prediction_values])
norm = BoundaryNorm(list(prediction_values)+[np.max(prediction_values)+1], cmap.N)
lc_map_filtered.plot.imshow(ax=axes[0], 
                   cmap=cmap,
                   norm=norm,
                   add_labels=True, 
                   add_colorbar=False,
                   interpolation='none')

# Plot classified image after applying rules
# reconstruct dataArray
lc_map_postproc=xr.DataArray(data=np_lc_map_postproc,dims=['y','x'],
                             coords={'y':lc_map.y.to_numpy(), 'x':lc_map.x.to_numpy()})
lc_map_postproc.plot.imshow(ax=axes[1], 
                   cmap=cmap,
                   norm=norm,
                   add_labels=True, 
                   add_colorbar=False,
                   interpolation='none')
# Remove axis on right plot
axes[1].get_yaxis().set_visible(False)
# add colour legend
axes[1].legend(patches_list, list(dict_map.keys()),
    loc='upper center', ncol =3, bbox_to_anchor=(0.5, -0.1))
# Add plot titles
axes[0].set_title('Classified Image Before Applying Rules')
axes[1].set_title('Classified Image After Applying Rules')

## Process all testing locations
Now let's process all three locations by simply copying and putting together all the post-processing steps, and iterating through all three locations:

In [ ]:
prediction_maps_postproc=[] # post-processed results
for i in range(0,len(prediction_maps)):
    lc_map=prediction_maps[i]
    # convert to numpy array
    np_lc_map=lc_map.squeeze().to_numpy()
    # majority filtering for a smoother classification map
    np_lc_map_postproc=modal(np_lc_map,footprint=disk(2),mask=np_lc_map!=0)
    # get geobox
    ds_geobox=lc_map.geobox
    # load and reproject hand layer
    hand=xr.open_dataset(hand_raster_tiles[i],engine="rasterio").squeeze()
    hand=xr_reproject(hand, ds_geobox, resampling="average")
    # convert to numpy array
    np_hand=hand.to_array().squeeze().to_numpy()
    # rasterise river network layer
    river_network_mask=xr_rasterize(gdf=river_network,
                                      da=lc_map.squeeze(),
                                      transform=ds_geobox.transform,
                                      crs=output_crs)
    # convert to numpy array
    np_river_network_mask=river_network_mask.to_numpy()
    # apply the rule
    np_lc_map_postproc[((np_lc_map==dict_map['Water body'])&(np_hand<=45))
                       |(np_river_network_mask==1)]=dict_map['Water body']
    # get bounding box
    bbox=ds_geobox.extent.boundingbox
    dc = datacube.Datacube(app='s2_geomedian')
    query_geomedian= {
        'time': ('2021'),
        'x': (bbox[0],bbox[2]),
        'y': (bbox[1],bbox[3]),
        'resolution':(-10, 10),
        'crs':output_crs,
        'output_crs': output_crs,
        'measurements':['green','swir_1']
    }
    ds_geomedian = dc.load(product="gm_s2_annual", **query_geomedian)
    ds_MNDWI = calculate_indices(ds=ds_geomedian, index='MNDWI', satellite_mission='s2',drop=True).squeeze()
    # reassign water using NDWI calculated from annual S2 geomedian
    np_MNDWI=ds_MNDWI['MNDWI'].to_numpy()
    np_lc_map_postproc[np_MNDWI>=0]=dict_map['Water body']

    # load and reproject google buildings raster
    google_buildings=xr.open_dataset(google_building_tiles[i],engine="rasterio").squeeze()
    google_buildings_mask=xr_reproject(google_buildings, ds_geobox, resampling="average")
    # convert to numpy array
    np_google_buildings_mask=google_buildings_mask.to_array().squeeze().to_numpy()
    # load and reproject WSF 2019 layer
    wsf2019=xr.open_dataset(wsf2019_raster_tiles[i],engine="rasterio").squeeze()
    wsf2019=xr_reproject(wsf2019, ds_geobox, resampling="nearest")
    # convert to numpy array
    np_wsf2019=wsf2019.to_array().squeeze().to_numpy()
    # apply rule
    np_lc_map_postproc[(np_google_buildings_mask==1)|(np_wsf2019==1)]=dict_map['Settlements']
    # buffer Settlements areas
    urban_buffered=binary_dilation(np_lc_map==dict_map['Settlements'],footprint=disk(5))
    # apply rule
    np_lc_map_postproc[(urban_buffered==1)&(np_lc_map==dict_map['Aquatic or regularly flooded herbaceous vegetation'])]=dict_map['Field crops']
    # rasterise road network layer
    road_network_mask=xr_rasterize(gdf=road_network,
                                  da=lc_map.squeeze(),
                                  transform=ds_geobox.transform,
                                  crs=output_crs)
    # convert to numpy
    np_road_network_mask=road_network_mask.to_numpy()
    # apply the rule
    np_lc_map_postproc[np_road_network_mask==1]=dict_map['Settlements']

    # rasterise shoreline layer
    shorelines_2021_mask=xr_rasterize(gdf=shorelines_gdf_2021,da=lc_map.squeeze(),
                                      transform=ds_geobox.transform,crs=output_crs) 
    # clip shoreline mask layers
    shorelines_2021_mask_tile=xr_reproject(shorelines_2021_mask, ds_geobox, resampling="nearest") 
    # convert to numpy
    np_shorelines_2021_mask=shorelines_2021_mask_tile.squeeze().to_numpy()
    # apply rule
    np_lc_map_postproc[(np_shorelines_2021_mask==0)&(np_lc_map==dict_map['Mangrove'])]=dict_map['Forest plantations']

    # reconstruct dataArray
    lc_map_postproc=xr.DataArray(data=np_lc_map_postproc,dims=['y','x'],
                             coords={'y':lc_map.y.to_numpy(), 'x':lc_map.x.to_numpy()})
    # set spatial reference
    lc_map_postproc.rio.write_crs(output_crs, inplace=True)
    # append to list
    prediction_maps_postproc.append(lc_map_postproc)

We can plot and compare all final post-processed results with initial predictions without post-processing, along with the satellite images for comparison:

In [ ]:
for i in range(0, len(prediction_maps)):
    fig, axes = plt.subplots(1, 3, figsize=(24, 8))

    # Plot true colour image
    rgb(rgb_images[i].to_dataset(dim='band'),bands=[1,2,3],
        ax=axes[0], percentile_stretch=(0.01, 0.99))
    
    # Plot initial classified image
    prediction_values=np.unique(prediction_maps[i])
    cmap=ListedColormap([colours[k] for k in prediction_values])
    norm = BoundaryNorm(list(prediction_values)+[np.max(prediction_values)+1], cmap.N)
    prediction_maps[i].plot.imshow(ax=axes[1], 
                   cmap=cmap,
                   norm=norm,
                   add_labels=True, 
                   add_colorbar=False,
                   interpolation='none')
    
    # Plot post-processed classified image
    prediction_maps_postproc[i].plot.imshow(ax=axes[2], 
                   cmap=cmap,
                   norm=norm,
                   add_labels=True, 
                   add_colorbar=False,
                   interpolation='none')
                   
    # Remove axis on middle and right plot
    axes[1].get_yaxis().set_visible(False)
    axes[2].get_yaxis().set_visible(False)
    # add colour legend
    patches_list=[Patch(facecolor=colour) for colour in colours.values()]
    axes[1].legend(patches_list, list(dict_map.keys()),
        loc='upper center', ncol =3, bbox_to_anchor=(0.5, -0.1))
    # Add plot titles
    axes[0].set_title('True Colour Geomedian')
    axes[1].set_title('Classified Image')
    axes[2].set_title('Classified Image - Postprocessed')

## Save as geotiff
One you finish the post-processing you can now export our post-processed results to sandbox disk as Cloud-Optimised GeoTIFFs:

In [ ]:
for i in range(0, len(prediction_maps_postproc)):
    write_cog(prediction_maps_postproc[i], 'Results/Mozambique_land_cover_prediction_postprocessed_2021_location_'+str(i)+'.tif', overwrite=True)